<a href="https://colab.research.google.com/github/ElenaGordienko1/NLP/blob/main/NLP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выпонение лабораторной работы 3

## 1 часть

Импорт библиотек ⭐

In [1]:
import numpy as np
import pandas as pd

### Функция линейной регрессии

In [2]:
def linear_regression(X: np.ndarray) -> float:

    weight = np.random.randn()
    bias = np.random.randn()

    return float(X * weight + bias)

### Функция активации: сигмоида





In [3]:
def activation_func(x: float) -> float:

    return 1 / (1 + np.exp(-x))

### Нейрон



In [4]:
def neuron(x: np.ndarray):

    temp_result = linear_regression(x)
    result = activation_func(temp_result)
    return result


### Применение на датасете

In [5]:
df = pd.read_csv("/content/Mall_Customers.csv")

X = df['Annual Income (k$)'].values
Y = df['Spending Score (1-100)'].values

x_sample = X[0]

output = neuron(x_sample)
print(f"Входное значение X: {x_sample}, Выход нейрона: {output}")

Входное значение X: 15, Выход нейрона: 8.824849190884547e-05


## 2 часть

### Импорт, активация, нормализация

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

def activation_func(x):
    return 1 / (1 + np.exp(-x))

def activation_derivative(x):
    return x * (1 - x)

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


### Класс с возможностью задать количество нейронов слоя и произвести обучение

In [7]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))
        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))
        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = activation_func(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = activation_func(self.z2)
        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = activation_func(self.z3)
        return self.a3

    def backward(self, X, y):
        m = X.shape[0]

        dZ3 = self.a3 - y
        dW3 = np.dot(self.a2.T, dZ3) / m
        db3 = np.sum(dZ3, axis=0, keepdims=True) / m

        dZ2 = np.dot(dZ3, self.W3.T) * activation_derivative(self.a2)
        dW2 = np.dot(self.a1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dZ1 = np.dot(dZ2, self.W2.T) * activation_derivative(self.a1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 100 == 0:
                loss = np.mean((y - self.a3) ** 2)
                print(f'Epoch {epoch}, Loss: {loss:.4f}')


### Проверка



In [8]:
df = pd.read_csv("/content/Mall_Customers.csv")

X_raw = df['Annual Income (k$)'].values.reshape(-1, 1)
Y_raw = df['Spending Score (1-100)'].values.reshape(-1, 1)

X_normalized = normalize(X_raw)
Y_normalized = normalize(Y_raw)

input_size = 1
hidden1_size = 5
hidden2_size = 5
output_size = 1
learning_rate = 0.01
epochs = 1000

nn = NeuralNetwork(input_size, hidden1_size, hidden2_size, output_size, learning_rate)
nn.train(X_normalized, Y_normalized, epochs)

predictions = nn.forward(X_normalized)

print("Настоящее значение:", Y_raw.flatten()[:5])
print("Предсказание:", (predictions * 100).flatten()[:5])

Epoch 0, Loss: 0.0692
Epoch 100, Loss: 0.0691
Epoch 200, Loss: 0.0691
Epoch 300, Loss: 0.0691
Epoch 400, Loss: 0.0691
Epoch 500, Loss: 0.0691
Epoch 600, Loss: 0.0691
Epoch 700, Loss: 0.0691
Epoch 800, Loss: 0.0691
Epoch 900, Loss: 0.0691
Настоящее значение: [39 81  6 77 40]
Предсказание: [50.20817834 50.20817834 50.20819002 50.20819002 50.2082017 ]


### Применение

In [9]:
def train_and_evaluate(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42
    )

    input_size = X_train.shape[1]
    hidden1_size = 5
    hidden2_size = 5
    output_size = 1
    learning_rate = 0.01
    epochs = 1000

    model = NeuralNetwork(input_size, hidden1_size, hidden2_size, output_size, learning_rate)
    model.train(X_train, Y_train, epochs)

    Y_train_pred = model.forward(X_train)
    Y_train_pred_binary = (Y_train_pred > 0.5).astype(int)
    Y_train_binary = (Y_train > 0.5).astype(int)
    train_accuracy = accuracy_score(Y_train_binary, Y_train_pred_binary)

    Y_test_pred = model.forward(X_test)
    Y_test_pred_binary = (Y_test_pred > 0.5).astype(int)
    Y_test_binary = (Y_test > 0.5).astype(int)
    test_accuracy = accuracy_score(Y_test_binary, Y_test_pred_binary)

    print(f"Точность на обучающей выборке: {train_accuracy:.4f}")
    print(f"Точность на тестовой выборке: {test_accuracy:.4f}")
    print("\n Отчет классификации на тестовой выборке:")
    print(classification_report(Y_test_binary, Y_test_pred_binary))

    return model

model = train_and_evaluate(X_normalized, Y_normalized)


Epoch 0, Loss: 0.0723
Epoch 100, Loss: 0.0722
Epoch 200, Loss: 0.0722
Epoch 300, Loss: 0.0722
Epoch 400, Loss: 0.0722
Epoch 500, Loss: 0.0722
Epoch 600, Loss: 0.0722
Epoch 700, Loss: 0.0722
Epoch 800, Loss: 0.0722
Epoch 900, Loss: 0.0722
Точность на обучающей выборке: 0.5000
Точность на тестовой выборке: 0.4250

 Отчет классификации на тестовой выборке:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.42      1.00      0.60        17

    accuracy                           0.42        40
   macro avg       0.21      0.50      0.30        40
weighted avg       0.18      0.42      0.25        40



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 3 часть

### Реализация GPT

In [12]:
import numpy as np

class Head:

    def __init__(self, embedding_dim, attention_dim, seq_len, drop_rate):
        self.embedding_dim = embedding_dim
        self.attention_dim = attention_dim
        self.seq_len = seq_len
        self.drop_rate = drop_rate

        self.key_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1
        self.query_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1
        self.value_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1

        self.mask_matrix = np.tril(np.ones((seq_len, seq_len)))

    def compute(self, input_tensor):
        B, T, C = input_tensor.shape

        key_proj = np.dot(input_tensor, self.key_matrix)    # (B, T, hs)
        query_proj = np.dot(input_tensor, self.query_matrix)  # (B, T, hs)

        attention_scores = np.matmul(query_proj, key_proj.transpose(0, 2, 1)) * (key_proj.shape[-1] ** -0.5)  # (B, T, T)
        attention_scores = np.where(self.mask_matrix[:T, :T] == 0, float('-inf'), attention_scores)  # (B, T, T)
        exp_scores = np.exp(attention_scores - np.max(attention_scores, axis=-1, keepdims=True))
        attention_weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)  # (B, T, T)

        dropout_mask = np.random.rand(*attention_weights.shape) > self.drop_rate
        attention_weights = attention_weights * dropout_mask

        value_proj = np.dot(input_tensor, self.value_matrix)  # (B, T, hs)
        output_tensor = np.matmul(attention_weights, value_proj)  # (B, T, T) @ (B, T, hs) -> (B, T, hs)

        return output_tensor

embedding_dim = 128
attention_dim = 32
seq_len = 64
drop_rate = 0.1

attention_head = Head(embedding_dim, attention_dim, seq_len, drop_rate)

batch_size = 4
time_steps = 16
channels = embedding_dim
input_data = np.random.randn(batch_size, time_steps, channels)

result = attention_head.compute(input_data)
print(result)


[[[-0.96816358  0.75093197 -1.08417396 ...  0.53711862  0.13297851
   -0.01480837]
  [-0.28324814 -0.1436078  -0.0991582  ...  0.74659862 -2.19628869
    0.02039824]
  [-0.04814912  0.01263255 -0.00774346 ...  0.00336655  0.00739636
    0.01384436]
  ...
  [ 0.07049248  0.6866702  -0.25333423 ... -1.09523656 -0.64059144
    0.09285656]
  [ 0.74155093 -0.09682823 -1.22513782 ... -1.0343882  -0.49598267
    0.07865018]
  [-0.43750086  0.4645297   0.46725375 ... -0.50024581  0.25711065
    0.35748516]]

 [[-0.53964632  1.1483524   1.01436077 ...  1.33296281  1.53151465
    0.66686896]
  [-0.30283081  0.64441557  0.56922411 ...  0.74801254  0.85943295
    0.37422375]
  [-0.09596716  0.13647349  0.18464583 ...  0.15471289  0.2610533
    0.17073862]
  ...
  [-0.17596198  0.25445193  0.16693777 ... -0.23102146  0.01967084
    0.13414608]
  [-0.07419337  0.33716137  0.22970324 ... -0.03382462  0.0094408
   -0.29776796]
  [-0.25506992 -0.17126381  0.00967381 ...  0.08799204 -0.52682791
   -0.02